In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings

from astropy.io import fits
from astropy.convolution import convolve_fft

#warnings.filterwarnings('ignore')
Band_name = [['FUV','GALEX_FUV'], ['Ha','Moffet_02.0'], ['IRAC8','IRAC_8.0']]
for band in Band_name:
    img_fits = fits.open('/Users/zhangl/desktop/SINGS/Img_Mon/NGC5194_{0}_flux.fits'.format(band[0]))
    img = img_fits[0].data
    psf_fits = fits.open('/Users/zhangl/desktop/SINGS/Kernel/Kernel_LoRes_{0}_to_Gauss_10.0.fits'.format(band[1]))
    psf = psf_fits[0].data
    img_cov = convolve_fft(img, psf, boundary = 'fill', allow_huge=True, preserve_nan=True)
    hdu = fits.PrimaryHDU(img_cov, header=img_fits[0].header)
    hdu.writeto('/Users/zhangl/desktop/SINGS/Img_Mon/NGC5194_{0}_cov2ten.fits'.format(band[0]), overwrite = True)
    print('Covolution of {0} image is OK!'.format(band[0]))

In [6]:
def Montage_Reproject(name, O2):
    import os
    import numpy as np
    from astropy.io import fits
    from astropy.table import Table


    Band_name = ['FUV', 'Ha', 'IRAC8']

    p_file = '/Users/zhangl/desktop/SINGS/Cube/{0}/{0}_DR5_{1}_cube.fits'.format(name.strip(),O2)
    if(os.path.isfile(p_file)==False):
        print('No such path or file:{0}'.format(p_file))
    p_fits = fits.open(p_file)
    dirs = '/Users/zhangl/desktop/SINGS/Img_Mon'
    if(os.path.exists(dirs) != True):
        os.makedirs(dirs)
    for band in Band_name:
        hdr = open('{1}/{0}_hdr.txt'.format(band, dirs),'w')
        rp_file = '/Users/zhangl/desktop/SINGS/Img_Mon/NGC5194_{0}_covLo.fits'.format(band)
        if(os.path.isfile(rp_file)==False):
            print('No such path or file:{0}'.format(rp_file))
        hdr.write('SIMPLE  = T\nBITPIX  = -64\nNAXIS   = 2\n')
        hdr.write('NAXIS1  = {0}\n'.format(str(p_fits[0].header['NAXIS1']*2+1)))
        hdr.write('NAXIS2  = {0}\n'.format(str(p_fits[0].header['NAXIS2']*20+19)))
        hdr.write('CTYPE1  = {0}\n'.format(p_fits[0].header['CTYPE1']))
        hdr.write('CTYPE2  = {0}\n'.format(p_fits[0].header['CTYPE2']))
        hdr.write('EQUINOX  = {0}\n'.format(p_fits[0].header['EQUINOX']))
        hdr.write('CRVAL1  = {0}\n'.format(p_fits[0].header['CRVAL1']))
        hdr.write('CRVAL2  = {0}\n'.format(p_fits[0].header['CRVAL2']))
        hdr.write('CRPIX1  = {0}\n'.format(str(p_fits[0].header['CRPIX1']*2)))
        hdr.write('CRPIX2  = {0}\n'.format(str(p_fits[0].header['CRPIX2']*20)))
        hdr.write('CDELT1  = {0}\n'.format(p_fits[0].header['CDELT1']))
        hdr.write('CDELT2  = {0}\n'.format(p_fits[0].header['CDELT2']))
#        hdr.write('PC1_1  = {0}\n'.format(p_fits[0].header['PC1_1']))
#        hdr.write('PC2_1  = {0}\n'.format(p_fits[0].header['PC2_1']))
#        hdr.write('PC1_2  = {0}\n'.format(p_fits[0].header['PC1_2']))
#        hdr.write('PC2_2  = {0}\n'.format(p_fits[0].header['PC2_2']))
        Ro2_s = np.arcsin(p_fits[0].header['PC2_1']*p_fits[0].header['CDELT2']/p_fits[0].header['CDELT1'])*180/np.pi
        Ro22_s = [round(Ro2_s-(Ro2_s/np.abs(Ro2_s)-1)*180,7), round(Ro2_s*180/np.abs(Ro2_s)-Ro2_s-(Ro2_s/np.abs(Ro2_s)-1)*180,7)]
        Ro2_c = np.arccos(p_fits[0].header['PC1_1'])*180/np.pi
        Ro22_c = [round(Ro2_c,7), round(360-Ro2_c,7)]
        Ro2 = list(set(Ro22_s).intersection(Ro22_c))[0]
        hdr.write('CROTA2  = {0}\n'.format(str(Ro2)))
        hdr.write('BUNIT  = mJy_sr^-1\n')
        hdr.write('END')
        hdr.close()
        os.system('mProject -f {1} {2}/NGC5194_{0}_Proj.fits {2}/{0}_hdr.txt'.format(band,rp_file,dirs))
        photd = fits.open('{1}/NGC5194_{0}_Proj.fits'.format(band, dirs), mode = 'update')
        photd[0].header['BUNIT'] = 'mJy/sr'
        photd.close()
    return

if __name__ == '__main__':

    gal_name = ['ngc5194']
    for name in gal_name:
        try:
            Montage_Reproject(name, 'LL2')
            print('Projection of {0}_iminging is OK!'.format(name.strip()))
        except:
            print('Something worry happens when reproject sample {0}!'.format(name.strip()))

No such path or file:/Users/zhangl/desktop/SINGS/Img_Mon/NGC5194_FUV_covLo.fits
No such path or file:/Users/zhangl/desktop/SINGS/Img_Mon/NGC5194_Ha_covLo.fits
No such path or file:/Users/zhangl/desktop/SINGS/Img_Mon/NGC5194_IRAC8_covLo.fits
Projection of ngc5194_iminging is OK!


In [3]:
import numpy as np 
from astropy.io import fits
from scipy.ndimage import rotate

def Circularization(path, mode, name, recover_edges, path1, path2):
    nm1 = name.split('.')[0]
    nm2 = name.split('.')[1]
    Img  = fits.open(path + '{0}_{3}/{1}_{2}.fits'.format(mode[0:2], nm1, nm2, path1))
    image = Img[0].data
    header = Img[0].header
    size_im = image.shape

    if (recover_edges == 1):
        sz = max(size_im)
        pixels_added_side = int((sz * np.sqrt(2.0) - sz)/ 2.0) + 1 
        pixels_added      = 2 * pixels_added_side
        new_image_size_x = size_im[0] + pixels_added
        new_image_size_y = size_im[1] + pixels_added

        new_image = np.zeros((new_image_size_x,new_image_size_y))
        image_one = np.zeros((new_image_size_x,new_image_size_y))

        new_image [pixels_added_side:pixels_added_side+size_im[0],pixels_added_side:pixels_added_side+size_im[1]] = image
        wh_data  = new_image != 0
        image_one[wh_data] = 1.0

        image = new_image
        header['CRPIX1']=(new_image_size_x-1)/2
        header['CRPIX2']=(new_image_size_y-1)/2
        header['NAXIS1']=new_image_size_x
        header['NAXIS2']=new_image_size_y

    size_im = len(image)
    center_pixel = int((size_im - 1) / 2)

    original_image = image

#First we mask the pixel outside the major circle contained into the image,
#since they are far away from the center an they do not contain useful information...

    xdist, ydist = np.mgrid[0:size_im, 0:size_im] - center_pixel
    distance_sq = xdist**2 + ydist**2 
############################################################################################################
    not_useful  = distance_sq > center_pixel**2
    image[not_useful] = 0
############################################################################################################

#now we rotate 180 deg te image and add it to itself, we rotate the composite 90 deg 
#and add it to itself, then 45 deg... in 10 ierations we have 16384 rotations...
    image_temp = image
    for iteration in range(14,0,-1):
        angle = float(360.0 / (2.0 ** iteration))
        rotate(image, angle, axes=(1, 0), reshape=False, output=image_temp)
        image = (image + image_temp )/2.0

    if (recover_edges == 1):
        image_temp = image_one
        for iteration in range(14,0,-1):
            angle = float(360.0 / (2.0 ** iteration))
            rotate(image_one, angle, axes=(1, 0), reshape=False, output=image_temp)
            image_one = (image_one + image_temp )/2.0

        wh = image_one > 0.1
        corrector = image_one * 0
        corrector[wh] = 1 / image_one[wh]
        image = image * corrector

    image[not_useful] = 0.0
    image /= image.sum()
    wh = np.abs(image) < 1.0e-10
    image[wh] = 0.0

    hdu = fits.PrimaryHDU(image,header)
    hdu.writeto(path + '{0}_{3}/{1}_{2}.fits'.format(mode[0:2], nm1, nm2, path2), overwrite = True)   

    difference = original_image - image
    difference[wh] = 0.0
    non_axi = np.abs(difference).sum() / np.abs(image).sum()
#print('------------------------------------------------------------------------------')
#print('The anisotropy parameter g of this circlularization is {0}'.format(str(round(non_axi,4))))
#print('The test was succesfully circularized!')
#print('------------------------------------------------------------------------------')
    return(non_axi)

In [4]:
import numpy as np
from astropy.io import fits
from numpy.fft import fft2, fftshift
from astropy.modeling.models import Gaussian2D
from photutils import create_matching_kernel, SplitCosineBellWindow
import matplotlib.pyplot as plt

def PSF_matching(path, mode, name, FWHM, path1, path2):
    nm1 = name.split('.')[0]
    nm2 = name.split('.')[1]
    Img  = fits.open(path + '{0}_{3}/{1}_{2}.fits'.format(mode[0:2], nm1, nm2, path1))
    psf = Img[0].data
    header = Img[0].header
    size_im = psf.shape
    scale = round(header['PIXSCALX'], 3)
    sigma = FWHM/(2*np.sqrt(2*np.log(2)))
    y, x = np.mgrid[0:round(size_im[0]*scale,3):scale, 0:round(size_im[0]*scale,3):scale] - round(scale*(size_im[0]-1)/2, 3)
    gm = Gaussian2D(100, 0, 0, sigma, sigma)
    gs = gm(x, y)
    gs /= gs.sum()
#    center_pixel = int((size_im - 1) / 2)
#    xdist, ydist = np.mgrid[0:size_im, 0:size_im] - center_pixel
#    distance_sq = xdist**2 + ydist**2 
#    not_useful  = distance_sq > center_pixel**2
#    gs[not_useful] = 0

    ftg1 = np.abs(np.real(fftshift(fft2(psf))))
    mask = ftg1 < 5*10**-3*np.max(ftg1)
    dist = np.sqrt(x**2 + y**2)
    kha = np.min(dist[mask])/scale
    alpha = 0.3*kha/((len(psf)-1)/2)
    beta = 0.7*kha/((len(psf)-1)/2)
    window = SplitCosineBellWindow(alpha, beta)
    kernel = create_matching_kernel(psf, gs, window=window)
    hdu = fits.PrimaryHDU(kernel, header)
    hdu.writeto(path + '{0}_{3}/{1}_{2}.fits'.format(mode[0:2], nm1, nm2, path2), overwrite = True)
    return(kha)

In [5]:
mode_list = ['SL2', 'SL1']
path1 = '/Users/zhangl/desktop/SINGS/Kernel/G_Aniano_Kernels/PSF4SL1/'
path2 = '/Users/zhangl/desktop/SINGS/Kernel/G_Aniano_Kernels/PSF4SL2/'
for mode in mode_list:
    wavef = open('/Users/zhangl/desktop/SINGS/Kernel/G_Aniano_Kernels/Wave/{0}_wavelength.txt'.format(mode))
    wavel = wavef.readlines()
    for one in wavel:
        try:
            PSF_matching(path1, mode, one.strip(), 4.5, 'PSFcirc', 'ker2GALEX4_5')
            Circularization(path1, mode, one.strip(), 0, 'ker2GALEX4_5', 'kercirc')
            PSF_matching(path2, mode, one.strip(), 6.4, 'PSFcirc', 'ker2MIPS6_4')
            Circularization(path2, mode, one.strip(), 0, 'ker2MIPS6_4', 'kercirc')
        except:
            print('Something worry happens when dealing with slice at wavelength {0}!'.format(one.strip()))

In [12]:
import numpy as np 
from astropy.io import fits
from scipy.ndimage import rotate

def CircKer(image, recover_edges):
    image = image
    size_im = image.shape

    if (recover_edges == 1):
        sz = max(size_im)
        pixels_added_side = int((sz * np.sqrt(2.0) - sz)/ 2.0) + 1 
        pixels_added      = 2 * pixels_added_side
        new_image_size_x = size_im[0] + pixels_added
        new_image_size_y = size_im[1] + pixels_added

        new_image = np.zeros((new_image_size_x,new_image_size_y))
        image_one = np.zeros((new_image_size_x,new_image_size_y))

        new_image [pixels_added_side:pixels_added_side+size_im[0],pixels_added_side:pixels_added_side+size_im[1]] = image
        wh_data  = new_image != 0
        image_one[wh_data] = 1.0

        image = new_image

    size_im = len(image)
    center_pixel = int((size_im - 1) / 2)

    original_image = image

#First we mask the pixel outside the major circle contained into the image,
#since they are far away from the center an they do not contain useful information...

    xdist, ydist = np.mgrid[0:size_im, 0:size_im] - center_pixel
    distance_sq = xdist**2 + ydist**2 
############################################################################################################
    not_useful  = distance_sq > center_pixel**2
    image[not_useful] = 0
############################################################################################################

#now we rotate 180 deg te image and add it to itself, we rotate the composite 90 deg 
#and add it to itself, then 45 deg... in 10 ierations we have 16384 rotations...
    image_temp = image
    for iteration in range(14,0,-1):
        angle = float(360.0 / (2.0 ** iteration))
        rotate(image, angle, axes=(1, 0), reshape=False, output=image_temp)
        image = (image + image_temp )/2.0

    if (recover_edges == 1):
        image_temp = image_one
        for iteration in range(14,0,-1):
            angle = float(360.0 / (2.0 ** iteration))
            rotate(image_one, angle, axes=(1, 0), reshape=False, output=image_temp)
            image_one = (image_one + image_temp )/2.0

        wh = image_one > 0.1
        corrector = image_one * 0
        corrector[wh] = 1 / image_one[wh]
        image = image * corrector

    image[not_useful] = 0.0
    image /= image.sum()
    wh = np.abs(image) < 1.0e-10
    image[wh] = 0.0
    return(image)

In [8]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.convolution import convolve_fft
from photutils.psf.matching import resize_psf

def Ker_resize(path, samfile, tagtfits, mode, pathker, pathkerf):
    wavef = open(samfile)
    wavel = wavef.readlines()
    img_fits = fits.open(tagtfits)
    for ii in range(0, len(wavel)):
        wave = wavel[ii].strip()
        nm1 = wave.split('.')[0]
        nm2 = wave.split('.')[1]
        kerf = fits.open(path + '{0}_{3}/{1}_{2}.fits'.format(mode[0:2], nm1, nm2, pathker))
        ker = kerf[0].data
        scalei = round(kerf[0].header['PIXSCALX'],3)
        scaleii = round(np.abs(img_fits[0].header['CDELT1'])*3600, 3)
        if(mode[0:2] == 'SL'):
            ker = ker[4:-4,4:-4]
# just make sure not to invoke the warning "the returned array has changed.", UserWarning
        rem = round((len(ker)*(scalei/scaleii))%1,2)
        if(rem >= 0.5):
            ind = int(round(rem/((2*scalei/scaleii)%1),2))
            if(ind != 0):
                ker = ker[ind:-ind,ind:-ind]
        ker = resize_psf(ker, scalei, scaleii, order=3)
        ker = CircKer(ker, 0)
        hdu = fits.PrimaryHDU(ker)
        hdu.writeto(path + '{0}_{3}/{1}_{2}.fits'.format(mode[0:2], nm1, nm2, pathkerf), overwrite = True)
        img_fits.close()
# The dimensions do not have to be odd in all directions, unlike in the non-fft convolve function
#        size = len(ker)
#        if ((size % 2) == 0):
#            size = size + 1
#            new_ker = np.zeros((size,size))
#            new_ker[0:size-1,0:size-1] = ker
#            ker = new_ker
    return ()

In [9]:
mode_list = ['SL2', 'SL1']
path1 = '/Users/zhangl/desktop/SINGS/Kernel/G_Aniano_Kernels/PSF4SL1/'
path2 = '/Users/zhangl/desktop/SINGS/Kernel/G_Aniano_Kernels/PSF4SL2/'
target = 'ngc5194'
for mode in mode_list:
    samfile = '/Users/zhangl/desktop/SINGS/Kernel/G_Aniano_Kernels/Wave/{0}_wavelength.txt'.format(mode)
    tagtfits = '/Users/zhangl/desktop/SINGS/Cube/{0}/{0}_DR5_{1}_cube.fits'.format(target, mode)
    try:
        Ker_resize(path1, samfile, tagtfits, mode, 'kercirc', 'ker4conv')
        Ker_resize(path2, samfile, tagtfits, mode, 'kercirc', 'ker4conv')
    except:
        print('Something worry happens when dealing with {0} data!'.format(mode))

In [10]:
import numpy as np
from astropy.io import fits
from astropy.convolution import convolve_fft

def Convolve(path, samfile, tagtfits, region, mode, dtype):
    wavef = open(samfile)
    wavel = wavef.readlines()
    img_fits = fits.open(tagtfits)
    img = img_fits[0].data
    img_conv = []
    for ii in range(0, len(wavel)):
        slice_Img = img[ii]
        wave = wavel[ii].strip()
        nm1 = wave.split('.')[0]
        nm2 = wave.split('.')[1]
        kerf = fits.open(path + '{0}_ker4conv/{1}_{2}.fits'.format(mode[0:2], nm1, nm2))
        ker = kerf[0].data
        img_cov = convolve_fft(slice_Img, ker, boundary = 'fill', allow_huge=True, preserve_nan=True)
        img_conv.append(list(img_cov))
    hdu = fits.PrimaryHDU(img_conv, header=img_fits[0].header)
    hdu.writeto(path + '{0}_{1}_{2}.fits'.format(mode, each, dtype), overwrite = True)
    return()

In [11]:
mode_list = ['SL2', 'SL1']
path1 = '/Users/zhangl/desktop/SINGS/Kernel/G_Aniano_Kernels/PSF4SL1/'
path2 = '/Users/zhangl/desktop/SINGS/Kernel/G_Aniano_Kernels/PSF4SL2/'
target = 'ngc5194'
region_list = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
for each in region_list:
    for mode in mode_list:
        samfile = '/Users/zhangl/desktop/SINGS/Kernel/G_Aniano_Kernels/Wave/{0}_wavelength.txt'.format(mode)
        tagtfits1 = '/Users/zhangl/desktop/SINGS/Cube/{0}/{0}_extranuc_{1}_DR5_{2}_cube.fits'.format(target, each, mode)
        tagtfits2 = '/Users/zhangl/desktop/SINGS/Cube/{0}/{0}_extranuc_{1}_DR5_{2}_cube_unc.fits'.format(target, each, mode)
        try:
            Convolve(path1, samfile, tagtfits1, each, mode, 'convolved')
            Convolve(path1, samfile, tagtfits2, each, mode, 'unc_convolved')
            Convolve(path2, samfile, tagtfits1, each, mode, 'convolved')
            Convolve(path2, samfile, tagtfits2, each, mode, 'unc_convolved')
        except:
            print('Something worry happens when dealing with region {0} data!'.format(each))